In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn.functional as F
from torch.utils.data import DataLoader


class Swish(nn.Module):
    def forward(self, x):
        return x * torch.sigmoid(x)

class AsReLU(nn.Module):
    """Asymmetric ReLU (example version, adjustable if you have your formula)"""
    def __init__(self, alpha=0.1):
        super().__init__()
        self.alpha = alpha
    def forward(self, x):
        return torch.maximum(x, self.alpha * x)


transform = transforms.Compose([
    transforms.Resize(224),  # Resize for pretrained models
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

# ===============================
# 3. Utility: Replace activation in model
# ===============================

def replace_activation(model, new_activation):
    for name, module in model.named_children():
        if isinstance(module, nn.ReLU):
            setattr(model, name, new_activation)
        else:
            replace_activation(module, new_activation)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_and_evaluate(model, epochs=3):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch [{epoch+1}/{epochs}] - Loss: {total_loss/len(trainloader):.4f}")

    # Evaluation
    model.eval()
    correct, total, test_loss = 0, 0, 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    accuracy = 100 * correct / total
    avg_loss = test_loss / len(testloader)
    return accuracy, avg_loss


activations = {
    "ReLU": nn.ReLU(),
    "Swish": Swish(),
    "AsReLU": AsReLU(alpha=0.1)
}

architectures = {
    "ResNet": models.resnet18(pretrained=False, num_classes=10),
    "MobileNet": models.mobilenet_v2(pretrained=False, num_classes=10)
}

results = []

for arch_name, arch in architectures.items():
    for act_name, act in activations.items():
        print(f"\n🔥 Training {arch_name} with {act_name} activation...")
        model = architectures[arch_name]
        replace_activation(model, act)
        acc, loss = train_and_evaluate(model, epochs=3)
        results.append((act_name, arch_name, acc, loss))
        print(f"✅ {act_name} on {arch_name}: Accuracy={acc:.2f}%, Loss={loss:.3f}")


print("\n=== Final Results (Approximate) ===")
print(f"{'Activation':<10} {'Architecture':<10} {'Accuracy':<10} {'Loss':<10}")
for act, arch, acc, loss in results:
    print(f"{act:<10} {arch:<10} {acc:.2f}% {loss:.3f}")

100%|███████████████████████████████████████████████████████████████████████████████| 170M/170M [00:43<00:00, 3.91MB/s]
C:\Users\Suraj\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Suraj\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)



🔥 Training ResNet with ReLU activation...
Epoch [1/3] - Loss: 1.3423
